In [1]:
using Revise
using IterativeSolvers
using LinearAlgebra

using Pkg
Pkg.activate("..")
using FSI

using LinearAlgebra

In [2]:
# construct a random block system

# block size
m = 20
n = 8
p = 10
q = 6

# operators
A = rand(m,m); A⁻¹ = inv(A)
B₁ᵀ = rand(m,n)
B₂ = rand(n,m)
M = rand(p,p)
G₁ᵀ = rand(p,q)
G₂ = rand(q,p)
T₁ᵀ = rand(p,n)
T₂ = rand(n,p)

# zero matrix
Omp = zeros(m,p)
Omq = zeros(m,q)
Onn = zeros(n,n)
Onq = zeros(n,q)
Opm = zeros(p,m)
Oqm = zeros(q,m)
Oqn = zeros(q,n)
Oqq = zeros(q,q)

# saddle point system
S = [A B₁ᵀ Omp Omq; B₂ Onn -T₂ Onq; Opm -T₁ᵀ M G₁ᵀ; Oqm Oqn G₂ Oqq]

# rhs
rċ = rand(m)
rf = rand(n)
ru̇ = rand(p)
rλ = rand(q)

b = [rċ;rf;ru̇;rλ];
b1 = [rċ;rf]
b2 = [ru̇;rλ];

In [3]:
# construct saddle point system
ċ = zeros(m)
f = zeros(n)
u̇ = zeros(p)
λ = zeros(q)
@time St = SaddleSystem1d((ċ, f, u̇, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

  1.390587 seconds (5.40 M allocations: 272.028 MiB, 6.09% gc time)


Saddle system with 8 constraints on fluid and 6 constraints on 1d body
   Fluid state of type Array{Float64,1}
   Fluid force of type Array{Float64,1}
   Body state of type Array{Float64,1}
   Joint force of type Array{Float64,1}


In [4]:
# Julia solver
@time x_theory = S\b

  1.818963 seconds (2.28 M allocations: 107.885 MiB, 2.90% gc time)


44-element Array{Float64,1}:
   8.14157163689804   
 -22.4272798913221    
  20.079840030821895  
  -5.194367063087652  
   6.016201319344597  
  -0.16083633064338138
 -10.48310090053501   
  10.53650159724268   
  -4.6317769055313045 
  -9.130735791658097  
  -4.927001689469608  
   3.350305886739693  
   3.3596700431331037 
   ⋮                  
  29.557520215113765  
 -30.71496064126173   
   3.9397666436757324 
 -30.793983810639652  
 -27.489259308593844  
  24.730413148664063  
  25.626425437748974  
  19.509142503683066  
  19.858302589953038  
 -21.507023835066725  
  -8.671483302694481  
   2.4405850014263155 

In [5]:
# my own solver
bt = (rċ, rf, ru̇, rλ);
aa,bb,cc,dd = St\bt
@time x_compute = [aa;bb;cc;dd]

  0.000005 seconds (6 allocations: 656 bytes)


44-element Array{Float64,1}:
   8.14157163690144   
 -22.427279891329547  
  20.07984003082812   
  -5.194367063089597  
   6.016201319347241  
  -0.16083633064452574
 -10.483100900538787  
  10.536501597246083  
  -4.63177690553211   
  -9.13073579166154   
  -4.927001689472384  
   3.3503058867413884 
   3.359670043135779  
   ⋮                  
  29.55752021512377   
 -30.714960641271787  
   3.939766643677297  
 -30.79398381065007   
 -27.489259308603238  
  24.730413148672223  
  25.6264254377587    
  19.50914250368932   
  19.858302589959038  
 -21.507023835074126  
  -8.671483302697538  
   2.440585001428242  

In [6]:
norm(x_theory-x_compute)

3.847241222304942e-11

### check step 1

In [7]:
[A B₁ᵀ;B₂ Onn]\b1
# norm([A B₁ᵀ;B₂ Onn]*x1 - [rċ;rf])

28-element Array{Float64,1}:
 -1.4285012826298233 
 -1.634131992842921  
  0.7716080578641569 
  0.4237100899570583 
 -2.2993325555188275 
  2.8690180799023435 
  1.3938473049806264 
  0.39680305645277203
 -0.08640361941692108
 -0.5186732084975988 
  4.056082550551316  
 -1.2744884595672266 
 -3.0752612263010737 
  ⋮                  
  3.8919845000016084 
  0.14042901445489364
  0.6825189674027863 
  0.7788703205832513 
 -1.0292994222233165 
  1.512762511981981  
  2.289840584953917  
  1.8249476147628287 
  1.1640782139161936 
 -0.1449407264510648 
 -2.047611290960835  
 -4.7459089984197655 

In [8]:
using ImportMacros
@import ViscousFlow.SaddlePointSystems as Sad

Sf = Sad.SaddleSystem((ċ,f),(A⁻¹,B₁ᵀ,B₂),tol=1e-3,
            issymmetric=false,isposdef=true,store=true,precompile=true)
x1,x2 = Sf\(b1[1:m], b1[m+1:end])

([-1.4285, -1.63413, 0.771608, 0.42371, -2.29933, 2.86902, 1.39385, 0.396803, -0.0864036, -0.518673, 4.05608, -1.27449, -3.07526, -2.31238, 0.0919275, -0.589987, 3.89198, 0.140429, 0.682519, 0.77887], [-1.0293, 1.51276, 2.28984, 1.82495, 1.16408, -0.144941, -2.04761, -4.74591])

### check step 2

In [9]:
x_theory[m+n+1:end]

16-element Array{Float64,1}:
  -7.660372123391573 
 -15.748394677359375 
  22.631859223050615 
  17.71002635177895  
  29.557520215113765 
 -30.71496064126173  
   3.9397666436757324
 -30.793983810639652 
 -27.489259308593844 
  24.730413148664063 
  25.626425437748974 
  19.509142503683066 
  19.858302589953038 
 -21.507023835066725 
  -8.671483302694481 
   2.4405850014263155

In [10]:
[M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ Oqq]\[b2[1:p]-T₁ᵀ*x2;b2[p+1:end]]

16-element Array{Float64,1}:
 -1.4497405094510518
 -1.6980843217623083
 -1.6867701616388513
  0.3357381306411476
  1.0977168820163599
  2.1400426682397775
  2.553416509658175 
  1.6499387904527718
 -1.7199200616749417
 -1.3463021596708096
 -1.1523616157498415
  1.1850641853728263
  6.537400107450067 
  0.4309055454229019
  0.8981972108734743
 -4.279317042387302 

In [11]:
x34 = gmres([M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ Oqq], [b2[1:p]-T₁ᵀ*x2;b2[p+1:end]], tol=1e-3)

16-element Array{Float64,1}:
 -1.4497405094510363 
 -1.698084321762121  
 -1.6867701616385236 
  0.33573813064136004
  1.0977168820163827 
  2.1400426682393388 
  2.553416509657894  
  1.649938790452513  
 -1.719920061674891  
 -1.3463021596706406 
 -1.1523616157494998 
  1.1850641853727943 
  6.537400107449531  
  0.4309055454226316 
  0.8981972108733396 
 -4.279317042386722  

In [12]:
cond([M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ Oqq])

9575.52309922449

### check step 3

In [13]:
x_theory[1:m+n]

28-element Array{Float64,1}:
   8.14157163689804   
 -22.4272798913221    
  20.079840030821895  
  -5.194367063087652  
   6.016201319344597  
  -0.16083633064338138
 -10.48310090053501   
  10.53650159724268   
  -4.6317769055313045 
  -9.130735791658097  
  -4.927001689469608  
   3.350305886739693  
   3.3596700431331037 
   ⋮                  
 -34.172229394074485  
  -3.9120906898285495 
  13.37391661479146   
 -10.500404482319249  
  -1.9867202744319057 
 -10.083100637992356  
  -7.224224680404563  
   7.638604323720963  
  -2.5395371875760184 
  18.291991685424563  
   5.3474208566974815 
  10.065970423816061  

In [14]:
ċ = x1 + A⁻¹*B₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂*x34[1:p]

20-element Array{Float64,1}:
  1.7406851943797617 
  1.9398658694685844 
  0.6407775853402442 
 -2.646559737136211  
  4.1897523319612    
 -6.26119547409236   
  0.47714430666509333
  1.5281044727473163 
  1.7550062930185066 
 -1.9062718898166815 
 -5.092142569486853  
  5.111200195312853  
  6.277675817747948  
  5.033686617916732  
 -1.7094638101172237 
  4.754683179390307  
 -6.793914923169452  
 -0.07814540699943462
 -0.6747551549330839 
 -6.7159549515425825 

In [15]:
f = x2 - inv(B₂*A⁻¹*B₁ᵀ)*T₂*x34[1:p]

8-element Array{Float64,1}:
  1.3593286140304712
 -6.566816997881373 
 -5.215764004328143 
  0.6901029509478298
 -1.0771915632534386
  2.6115252301846743
  3.165386205580694 
  4.680593173268031 